In [10]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install python-dotenv

In [3]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()
openai_key = os.environ.get('OPENAI_API_KEY')

client = OpenAI(
    api_key = openai_key
)

In [8]:
class MyAssistant:
    def __init__(self, openai_client, name = "Miriko"):
        self.name = name
        self.messages = [
            {"role": "system", "content": "You are a personal AI Assistant that helps user with their daily tasks."},
        ]
        self.client = openai_client
        print(self.greet())

    def greet(self):
        # print(self.chat("Say Greeting"))
        return f"Hello, {self.name} here. Anything i could help today?"

    # promt -> input user
    # system_content -> role the ai that we want
    # model -> used gpt model
    # stream -> to display the result in real time (like typing)
    def chat(self, prompt, system_content = None, model = "gpt-3.5-turbo", stream = True):
        system_content = system_content if system_content != None else "You are a personal AI Assistant that helps user with their daily tasks."
            
        self.messages.append({"role": "system", "content": system_content}) 
        
        self.messages.append({"role": "user", "content": prompt})
        
        if stream:
            completion = self.client.chat.completions.create(
                model = model,
                messages = self.messages,
                stream = True
            )
            for chunk in completion:
                if chunk.choices[0].delta.content is not None:
                    print(chunk.choices[0].delta.content, end="")
        else: 
            completion = client.chat.completions.create(
                model = model,
                messages = self.messages
            )
            print(completion.choices[0].message.content)
            
    def summarizer(self, content):
        return self.chat(content, system_content="""
                        You are a summarizer AI that summarizes the given content in two sentences.
                        The output format should be in a list
                    """.strip())

    def translator(self, content):
        return self.chat(content, system_content="""
                        You are a translator AI that translates the given content to user's requested language.
                        Don't make assumptions about what language the user wants to translate to.
                        Ask for clarification if a user doesn't specify a language.
                    """.strip())
        
    def english_grammar_master(self, content):
        return self.chat(content, system_content="""
                        You are an English grammar master that corrects the given content.
                    """.strip())
        
    def display_chat_history(self):
        for message in self.messages:
            print(message["role"], ":", message["content"])

In [9]:
my_assistant = MyAssistant(openai_client=client)

Hello, Miriko here. Anything i could help today?


In [10]:
my_assistant.chat("i want to say hello, how are you? in another language")

Sure! In which language would you like to learn how to say "hello, how are you?"?

In [11]:
my_assistant.chat("in japanese")

In Japanese, "hello, how are you?" can be translated as "こんにちは、お元気ですか？" (Konnichiwa, ogenki desu ka?).

In [12]:
my_assistant.display_chat_history()

system : You are a personal AI Assistant that helps user with their daily tasks.
system : You are a personal AI Assistant that helps user with their daily tasks.
user : i want to say hello, how are you? in another language
system : You are a personal AI Assistant that helps user with their daily tasks.
user : in japanese
